In [1]:
import cv2
import numpy as np
import torch

In [ ]:
!gdown 1aZfni62SjGzjkvsVBy8ZkX1IeTFgIecn

In [2]:
model = torch.hub.load('yolov5', 'custom', path='football-det-best.pt', source='local')
model.conf = 0.7

/bin/sh: 1: git: not found
YOLOv5 🚀 2023-11-29 Python-3.10.13 torch-2.1.1 CUDA:0 (NVIDIA A30, 24236MiB)

Fusing layers... 
Model summary: 212 layers, 20865057 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [3]:
%cd BoT-SORT

/home/samateen/football-match-tracker/BoT-SORT


In [4]:
class TrackerArgs:
    track_high_thresh: float = 0.5
    track_low_thresh: float = 0.1
    new_track_thresh: float = 0.6
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 1.6
    min_box_area: float = 10
    mot20: bool = False
    appearance_thresh: float = 0.25
    proximity_thresh = 0.5
    with_reid = False
    cmc_method = "sparseOptFlow"
    name = "exp"
    ablation = False

classes = {
    0: 'ball',
    1: 'goalie',
    2: 'player',
    3: 'referee'
}

In [5]:
from tracker.mc_bot_sort import BoTSORT

args = TrackerArgs()

# Initialize Tracker
tracker = BoTSORT(args)

cap = cv2.VideoCapture('../test_video.mp4')

# Get video frame size and fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'XVID'
out = cv2.VideoWriter('inference-output.mp4', fourcc, fps, (frame_width, frame_height))

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break

    # Use YOLO model to detect objects
    results = model(frame)

    # Parse results to get bounding boxes and confidence scores
    bboxes = results.xyxy[0][:, :4]  # xyxy format: (x1, y1, x2, y2)
    scores = results.xyxy[0][:, 4]

    img_size = frame.shape[:2]  # (height, width)
    img_info = (frame_height, frame_width)  # original image size

    # Use Tracker to track objects
    tracked_objects = tracker.update(results.xyxy[0].cpu(), frame)

    # Draw tracked objects on the frame
    for t in tracked_objects:
        tlwh = t.tlwh
        tid = t.track_id
        tcls = int(t.cls)
        vertical = tlwh[2] / tlwh[3] > args.aspect_ratio_thresh
        if tlwh[2] * tlwh[3] > args.min_box_area and not vertical:
            box = (int(tlwh[0]), int(tlwh[1]), int(tlwh[2]), int(tlwh[3]))
            cv2.rectangle(frame, box, (255, 0, 0), 2)
            cv2.putText(frame, classes[tcls], (int(tlwh[0]), int(tlwh[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # Write the frame into the file 'output.mp4'
    out.write(frame)

# Release everything after the job is finished
cap.release()
out.release()